### Count Distinct Using SQL as Robustness Check

In [1]:
!pip install sqlalchemy

In [1]:
import pandas as pd
from glob import glob
import libarchive # `pip install libarchive-c` and using with libarchive 3.4.x
from io import BytesIO
import re
from sqlalchemy import create_engine

In [2]:
engine = create_engine('sqlite:////opt/data/secc/raw.db', echo=False)

In [4]:
total_size = 0
for fn in sorted(glob('../secc/secc*.7z')):
    print(fn)
    with libarchive.file_reader(fn) as e:
        for entry in e:
            total_size += entry.size
            print(total_size / (1024*1024))
            with BytesIO() as f:
                for i, block in enumerate(entry.get_blocks()):
                    f.write(block)
                f.seek(0)
                #df = pd.read_csv(f, nrows=10)
                df = pd.read_csv(f)
                del df['Unnamed: 0']
                df.to_sql('secc', if_exists='append', con=engine)
    #break

../secc/secc_01_clean.7z
176.7793779373169


/opt/venv37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (7,8,10,11,12,13,19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


../secc/secc_02_clean.7z
461.01110553741455
../secc/secc_03_clean.7z
708.2516670227051
../secc/secc_04_clean.7z
1086.6022787094116
../secc/secc_05_clean.7z
2518.407386779785


/opt/venv37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


../secc/secc_06_clean.7z
2860.2195234298706


KeyboardInterrupt: 

In [5]:
engine.execute("SELECT count(*) FROM secc").fetchall()

[(11578233,)]

In [3]:
df = pd.read_sql('SELECT * from secc limit 10', engine)
df

,index,state,district,tehsil,panchayat,language,auto_inclusion_deprivation_or_exclusion_or_other,head_of_hh,gender,age,social_cat,fathers_and_mothers_name,deprivation_count,auto_inclusion_deprivation_code,total_members,hh_summary_auto_inclusion,hh_summary_auto_exclusion,hh_summary_auto_other,hh_summary_deprivation,village
0,0,ANDAMAN & NICOBAR ISLANDS,Nicobars,Car Nicobar,Arong(Village),eng,Auto Inclusion or Deprivation,None,None,None,None,None,None,None,None,None,None,None,None,None
1,1,ANDAMAN & NICOBAR ISLANDS,Nicobars,Car Nicobar,Arong(Village),eng,Auto Exclusion,None,None,None,None,None,None,None,None,None,None,None,None,None
2,2,ANDAMAN & NICOBAR ISLANDS,Nicobars,Car Nicobar,Arong(Village),eng,Others,None,None,None,None,None,None,None,None,None,None,None,None,None
3,3,ANDAMAN & NICOBAR ISLANDS,Nicobars,Car Nicobar,Big Lapati(Village),eng,Auto Inclusion or Deprivation,None,None,None,None,None,None,None,None,None,None,None,None,None
4,4,ANDAMAN & NICOBAR ISLANDS,Nicobars,Car Nicobar,Big Lapati(Village),eng,Auto Exclusion,None,None,None,None,None,None,None,None,None,None,None,None,None
5,5,ANDAMAN & NICOBAR ISLANDS,Nicobars,Car Nicobar,Big Lapati(Village),eng,Others,None,None,None,None,None,None,None,None,None,None,None,None,None
6,6,ANDAMAN & NICOBAR ISLANDS,Nicobars,Car Nicobar,CAR NICOBAR TRIBAL COUNCIL(GP),eng,Auto Inclusion or Deprivation,None,None,None,None,None,None,None,None,None,None,None,None,None
7,7,ANDAMAN & NICOBAR ISLANDS,Nicobars,Car Nicobar,CAR NICOBAR TRIBAL COUNCIL(GP),eng,Auto Exclusion,None,None,None,None,None,None,None,None,None,None,None,None,None
8,8,ANDAMAN & NICOBAR ISLANDS,Nicobars,Car Nicobar,CAR NICOBAR TRIBAL COUNCIL(GP),eng,Others,None,None,None,None,None,None,None,None,None,None,None,None,None
9,9,ANDAMAN & NICOBAR ISLANDS,Nicobars,Car Nicobar,Chuckchucha(Village),eng,Auto Inclusion or Deprivation,None,None,None,None,None,None,None,None,None,None,None,None,None


In [4]:
df.columns

Index(['index', 'state', 'district', 'tehsil', 'panchayat', 'language',
       'auto_inclusion_deprivation_or_exclusion_or_other ', 'head_of_hh',
       'gender', 'age', 'social_cat', 'fathers_and_mothers_name',
       'deprivation_count', 'auto_inclusion_deprivation_code', 'total_members',
       'hh_summary_auto_inclusion', 'hh_summary_auto_exclusion',
       'hh_summary_auto_other', 'hh_summary_deprivation', 'village'],
      dtype='object')

In [14]:
df = pd.read_sql('SELECT state, `auto_inclusion_deprivation_or_exclusion_or_other `, count(distinct head_of_hh) from secc group by state, `auto_inclusion_deprivation_or_exclusion_or_other `', engine)
df

,state,auto_inclusion_deprivation_or_exclusion_or_other,count(distinct head_of_hh)
0,ANDAMAN & NICOBAR ISLANDS,Auto Exclusion,0
1,ANDAMAN & NICOBAR ISLANDS,Auto Inclusion or Deprivation,0
2,ANDAMAN & NICOBAR ISLANDS,Others,0
3,ANDHRA PRADESH,Auto Exclusion,0
4,ANDHRA PRADESH,Auto Inclusion or Deprivation,0
5,ANDHRA PRADESH,Others,0
6,ARUNACHAL PRADESH,Auto Exclusion,172612
7,ARUNACHAL PRADESH,Auto Inclusion or Deprivation,172612
8,ARUNACHAL PRADESH,Others,172612
9,ASSAM,Auto Exclusion,1352624
